In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'crime-cast-forecasting-crime-categories:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F77420%2F8446444%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T044025Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8ced181d46c490bda48dd8d17691b5681b7fd41fd22f87f95afba676cb791c80ef96bc6ff68de7a7feb80ff43009d6258d5f9d7de251e41e524bf7a084feb84cd6e7b53c8871022ee2edf9589ba1934dc338513cf1a6827e25edba77efc58cc3baf5cc1fe3ef8e962832e412441086be79a42d48f28178a5d3c191698c5093e825ecb25cf86bff4747d84b659e372fcadee92e7c0aaaa4e6e465d40d4d19b126a50679b27177bdbc6ea620fde0ecebde33ef41fd905a102532ba8445e48a4363c9cd00dc5c25e3b86d450d34c1592155c8ed7997d0037b88b397e6d7d50c89b37d0cba8ad8d907b184073843d9fe3813859dc60692b2b63fb0ee7bed18840f18'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xgboost > /dev/null 2>&1
!pip install plotly > /dev/null 2>&1
!pip install lightgbm > /dev/null 2>&1
!pip install imblearn > /dev/null 2>&1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsOneClassifier,OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from random import *
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go

# Data Loading

In [ ]:
data = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
X_test = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')
X=data.copy()

# Exploratory Data Analysis

* ## Data Summary

In [ ]:
print(f"Number of rows: {X.shape[0]}")
print(f"Number of columns: {X.shape[1]}")
print("\nData types of each feature:")
print(X.dtypes)

* ## Missing Values

- This bar chart visualizes the number of missing values for each feature in the dataset, highlighting that ‘Cross_Street’, ‘Weapon_Used_Code’, and ‘Weapon_Description’ have the highest counts of missing data.

In [ ]:

print("\nMissing Values Analysis:")
missing_values = X.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(missing_values)


fig = px.bar(missing_values, x=missing_values.index, y=missing_values.values,
             labels={'index': 'Features', 'y': 'Number of Missing Values'},
             title="Missing Values Count by Feature")
fig.show()

* ## Data Summary

1. This summary highlights the central tendency and spread of key numerical features in the dataset. Notable observations include a wide range of dates for Date_Reported, variability in Victim_Age, and significant missing data in Weapon_Used_Code.

In [ ]:
X.describe()

In [ ]:
l = LabelEncoder()
label=X['Crime_Category']
labeldf = pd.DataFrame(l.fit_transform(label),columns=['Crime_Category'])

numerical = X.select_dtypes(include=['int64','float64','int32']).columns
categorical = X.select_dtypes(include=['object']).columns

numericaldf = pd.concat([X[numerical], labeldf], axis=1)

* ## Feature Distributions & Outlier Detection

In [ ]:
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns

for column in numerical_columns:
    fig = px.histogram(X, x=column, nbins=50, marginal='box', 
                       title=f'Distribution of {column}', 
                       labels={column: column})
    fig.show()

## Correlation Matrix

* Higly correlated features: 
1. Latittude and Longitude
2. Hour Occurred & Time Occurred
3. Area ID and Reporting Dist No

In [ ]:
correlation_matrix = numericaldf.corr()

fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1,
                )

fig.update_layout(title='Correlation Matrix for Numerical Features')

pio.show(fig)

* ## Target Variable Analysis

- This bar chart illustrates the distribution of crime categories within the dataset, revealing that ‘Property Crimes’ are the most common, followed by ‘Violent Crimes,’ while categories like ‘Other Crimes’ and ‘Crimes against Persons’ are significantly less frequent.

In [ ]:
fig = px.histogram(X, x='Crime_Category', 
                   title='Distribution of Crime Categories', 
                   labels={'Crime_Category': 'Crime Category'},
                   nbins=len(X['Crime_Category'].unique()))
fig.show()

* ## Crimes reported by year and month

In [ ]:
X['Date_Reported'] = pd.to_datetime(X['Date_Reported'])
X['Date_Occurred'] = pd.to_datetime(X['Date_Occurred'])

X['Year_Reported'] = X['Date_Reported'].dt.year
X['Month_Reported'] = X['Date_Reported'].dt.month


plt.figure(figsize=(10, 6))
sns.histplot(data=X, x='Year_Reported', bins=len(X['Year_Reported'].unique()), kde=False)
plt.title('Crime Reports by Year')
plt.xlabel('Year Reported')
plt.ylabel('Number of Reports')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()


plt.figure(figsize=(10, 6))
sns.histplot(data=X, x='Month_Reported', bins=12, kde=False)
plt.title('Crime Reports by Month')
plt.xlabel('Month Reported')
plt.ylabel('Number of Reports')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

## Pairplot among features

In [ ]:
plt.figure()
sns.pairplot(X, diag_kind="hist")
plt.show()

# Data Preprocessing

1. ## Changing of Date and time to respective Month, Year and Day Occurred and Reported.

In [ ]:
date_format = '%m/%d/%Y %I:%M:%S %p'  # Month/Day/Year Hour:Minute:Second AM/PM

# train.csv
X['Date_Reported'] = pd.to_datetime(X['Date_Reported'], format=date_format)
X['Date_Occurred'] = pd.to_datetime(X['Date_Occurred'], format=date_format)


X['Year_Reported'] = X['Date_Reported'].dt.year
X['Month_Reported'] = X['Date_Reported'].dt.month
X['Day_Reported'] = X['Date_Reported'].dt.day
X['Hour_Occurred'] = X['Time_Occurred'].apply(lambda x: int(x // 100))

# test.csv
X_test['Date_Reported'] = pd.to_datetime(X_test['Date_Reported'], format=date_format)
X_test['Date_Occurred'] = pd.to_datetime(X_test['Date_Occurred'], format=date_format)

X_test['Year_Reported'] = X_test['Date_Reported'].dt.year
X_test['Month_Reported'] = X_test['Date_Reported'].dt.month
X_test['Day_Reported'] = X_test['Date_Reported'].dt.day
X_test['Hour_Occurred'] = X_test['Time_Occurred'].apply(lambda x: int(x // 100))

X['Day_Diff'] = abs((X['Date_Reported'] - X['Date_Occurred']).dt.days)
X_test['Day_Diff'] = abs((X_test['Date_Reported'] - X_test['Date_Occurred']).dt.days)

## **Dropping of columns**

In [ ]:
y=X['Crime_Category']
drop_columns=['Location', 'Cross_Street', 'Date_Reported', 'Date_Occurred','Area_Name',
              'Weapon_Description','Longitude','Reporting_District_no','Hour_Occurred',
              'Status_Description','Premise_Description']

le=LabelEncoder()
y_coded=le.fit_transform(y)
y_coded=pd.DataFrame(y_coded,columns=['Crime_Category']).values.ravel()
X.columns

In [ ]:
X.drop(columns=drop_columns,inplace=True)
X.drop(columns=['Crime_Category'],inplace=True)
X.columns

## Handling null values for Victim_age

In [ ]:
X_test=pd.DataFrame(X_test)
X_test.drop(columns=drop_columns,inplace=True)
X['Victim_Age']=X['Victim_Age'].replace(-1,np.nan)
X['Victim_Age']=X['Victim_Age'].replace(-2,np.nan)

X_test['Victim_Age']=X_test['Victim_Age'].replace(-1,np.nan)
X_test['Victim_Age']=X_test['Victim_Age'].replace(-2,np.nan)

In [ ]:
class LabelEncodingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.le = LabelEncoder()
        self.le.fit(X)
        return self

    def transform(self, X):
        return self.le.transform(X).reshape(-1, 1)

In [ ]:
X_test.columns

## Modus Operandi 

* **The values are split by unique values, and one hot is performed for each entries**

In [ ]:
X['Modus_Operandi'] = X['Modus_Operandi'].fillna('null')
X_test['Modus_Operandi'] = X_test['Modus_Operandi'].fillna('null')


modus_operandi_list = X['Modus_Operandi'].str.split()
unique_modus_operandi = set()
for row in modus_operandi_list:
    unique_modus_operandi.update(row)
unique_modus_operandi = sorted(unique_modus_operandi)

# print("Unique Modus_Operandi codes from training data:", unique_modus_operandi)

In [ ]:
def feature_engineer_modus_operandi(df, unique_modus_operandi):
    modus_operandi_list = df['Modus_Operandi'].str.split()

    modus_operandi_cols = {f'Modus_{modus}': [] for modus in unique_modus_operandi}

    for row in modus_operandi_list:
        row_set = set(row)
        for modus in unique_modus_operandi:
            modus_operandi_cols[f'Modus_{modus}'].append(1 if modus in row_set else 0)

    modus_operandi_df = pd.DataFrame(modus_operandi_cols)

    df = pd.concat([df, modus_operandi_df], axis=1)

    return df
X = feature_engineer_modus_operandi(X, unique_modus_operandi)
X_test = feature_engineer_modus_operandi(X_test, unique_modus_operandi)
X.drop(columns=['Modus_Operandi'],inplace=True)
X_test.drop(columns=['Modus_Operandi'],inplace=True)

In [ ]:
X.shape

## Feature Imputation and Encoding for Numerical & Categorical Features

In [ ]:
numerical_columns = ['Latitude', 'Time_Occurred', 'Victim_Age','Part 1-2']
categorical_columns = ['Victim_Descent', 'Status','Victim_Sex']
area_id_column = ['Area_ID']
weapon_code_column=['Weapon_Used_Code']
premise_code_column=['Premise_Code']

#Numerical Variables
numerical_transformer = ColumnTransformer(
    transformers=[
        ('latitude', Pipeline([
            ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
            ('scaler', StandardScaler())
        ]), ['Latitude']),
        ('time_occurred', Pipeline([
            ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
            ('scaler', StandardScaler())
        ]), ['Time_Occurred']),
        ('victim_age', Pipeline([
            ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
            ('scaler', StandardScaler())
        ]), ['Victim_Age']),
        ('part',Pipeline([
            ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
            ('scaler', StandardScaler())
        ]), ['Part 1-2']),
    ]
)

#Categorical Variables
categorical_transformer = ColumnTransformer(
    transformers=[
        ('victim_descent', Pipeline([
            ('imputer', SimpleImputer(strategy='constant',fill_value='vd_null')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['Victim_Descent']),
        ('status', Pipeline([
            ('imputer', SimpleImputer(strategy='constant',fill_value='status_null')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['Status']),
    ]
)

area_id_transformer = Pipeline(steps=[
    ('imputer',KNNImputer(n_neighbors=3, weights="uniform")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


premise_code_transformer= Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

weapon_code_transformer= Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3, weights="uniform")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

passthrough_pipeline=Pipeline(
    steps=[]
)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('area', area_id_transformer, area_id_column),
        ('weapon_code',weapon_code_transformer,weapon_code_column),
        ('premise_code',premise_code_transformer,premise_code_column)
    ],
    remainder='passthrough'

)

In [ ]:
X_processed = preprocessor.fit_transform(X)
X_data=pd.DataFrame(X_processed)

## Feature Engineering for Victim_Sex

- **Prediction of Victim_Sex using SVC based on Age and Premise Code**

In [ ]:
label_encoder = LabelEncoder()
X['Victim_Sex_encoded'] = label_encoder.fit_transform(X['Victim_Sex'].astype(str))

train_X = X[X['Victim_Sex'].notna()]
test_X = X[X['Victim_Sex'].isna()]

X_train_svc = train_X[['Victim_Age', 'Premise_Code']]
y_train_svc = train_X['Victim_Sex_encoded']

imputer = KNNImputer(n_neighbors=3, weights="uniform")
X_train_svc['Victim_Age'] = imputer.fit_transform(X_train_svc[['Victim_Age']])

svc = SVC()
svc.fit(X_train_svc, y_train_svc)

X_test_svc = test_X[['Victim_Age', 'Premise_Code']]
X_test_svc['Victim_Age'] = imputer.transform(X_test_svc[['Victim_Age']])

test_X['Victim_Sex_encoded'] = svc.predict(X_test_svc)

final_X = pd.concat([train_X, test_X], axis=0).sort_index()



In [ ]:
X_data['Victim_Sex'] = final_X['Victim_Sex_encoded']

In [ ]:
X_test_vs=X_test.drop(columns=['Victim_Sex'])

In [ ]:
vs_feature=X_test[['Victim_Age', 'Premise_Code']]
vs_feature['Victim_Age'] = imputer.transform(vs_feature[['Victim_Age']])

In [ ]:
X_test.columns

In [ ]:
X_test=preprocessor.transform(X_test)


In [ ]:
X_test=pd.DataFrame(X_test)

In [ ]:
t = svc.predict(vs_feature)
X_test['Victim_Sex']=t

In [ ]:
print(f"Count of NaN values in 'train_csv': {X_data.isna().sum().sum()} values")
print(f"Count of NaN values in 'test_csv': {X_test.isna().sum().sum()} values")

In [ ]:
X_data

## SMOTE
- Oversampling of X and y variables using SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.pipeline import Pipeline
from collections import Counter
from lightgbm import LGBMClassifier


smote = SMOTE(sampling_strategy={0:2000,3: 2000, 1:2000, 2:2000}, random_state=42)

# smote = SMOTE(random_state=42)

X_data.columns = X_data.columns.astype(str)
X_resampled, y_resampled = smote.fit_resample(X_data, y_coded)

print(f"Original class distribution: {Counter(y_coded)}")
print(f"Resampled class distribution: {Counter(y_resampled)}")

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_resampled,y_resampled,stratify=y_resampled,test_size=0.3,random_state=42)

In [ ]:
X_test.columns=X_test.columns.astype(str)
X_train.columns=X_train.columns.astype(str)
X_val.columns=X_val.columns.astype(str)

In [ ]:
X_test.columns

In [ ]:
X_data.shape

# Hyperparameter Tuning


In [ ]:
# classifiers = {
#     'RandomForest': (RandomForestClassifier(random_state=42), {
#         'classifier__n_estimators': randint(50, 200),
#         'classifier__max_depth': randint(3, 10),
#         'classifier__min_samples_split': randint(2, 10),
#         'classifier__min_samples_leaf': randint(1, 10)
#     }),
#     'LogisticRegression': (LogisticRegression(max_iter=500, random_state=42, solver='liblinear'), {
#         'classifier__C': uniform(0.01, 10),
#         'classifier__penalty': ['l1', 'l2']
#     }),
#     'XGBoost': (XGBClassifier(random_state=42), {
#         'classifier__n_estimators': randint(50, 200),
#         'classifier__max_depth': randint(3, 10),
#         'classifier__learning_rate': uniform(0.01, 0.3),
#         'classifier__subsample': uniform(0.7, 0.3),
#         'classifier__colsample_bytree': uniform(0.7, 0.3)
#     }),
#     'SVC': (SVC(probability=True, random_state=42), {
#         'classifier__C': uniform(0.1, 10),
#         'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#         'classifier__gamma': ['scale', 'auto']
#     })
# }


In [ ]:
# for name, (classifier, param_grid) in classifiers.items():
#     print(f"\nTuning hyperparameters for {name}...\n")

#     # Define the pipeline with the preprocessor, PCA, and the classifier
#     pipeline = Pipeline(steps=[
#         ('pca', PCA(n_components=0.95)),  # Keep 95% of the variance
#         ('classifier', classifier)
#     ])

#     # Perform RandomizedSearchCV
#     random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=50, cv=3, verbose=2, n_jobs=-1, random_state=42)
#     random_search.fit(X_train, y_train)

#     # Get the best estimator
#     best_pipeline = random_search.best_estimator_

#     # Predict on the validation set
#     y_pred = best_pipeline.predict(X_val)

#     # Evaluate the model
#     print(f"Best parameters for {name}: ", random_search.best_params_)
#     print(classification_report(y_val, y_pred))

### **Values of HPT**

**RandomForestClassifier:**
- `random_state=42`
- `max_depth=9`
- `min_samples_leaf=1`
- `min_samples_split=5`
- `n_estimators=145`
- `class_weight='balanced'`

**XGBClassifier:**
- `random_state=42`
- `colsample_bytree=0.8532241907732696`
- `learning_rate=0.1352233009446337`
- `max_depth=7`
- `n_estimators=100`
- `subsample=0.7808237001395564`
- `scale_pos_weight=1`

**HistGradientBoostingClassifier:**
- `random_state=42`
- `learning_rate=0.1`
- `max_iter=100`
- `class_weight='balanced'`

**LGBMClassifier:**
- `random_state=42`
- `learning_rate=0.1`
- `n_estimators=100`
- `class_weight='balanced'`

**SVC:**
- `random_state=42`
- `C=0.5522728891053806`
- `gamma='auto'`
- `kernel='linear'`
- `class_weight='balanced'`

# Model Selection

In [ ]:
clf1 = RandomForestClassifier(
    random_state=42,
    max_depth=9,
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=145,
    class_weight='balanced'
)

In [ ]:
clf2 = XGBClassifier(
    random_state=42,
    colsample_bytree=0.8532241907732696,
    learning_rate=0.1352233009446337,
    max_depth=7,
    n_estimators=100,
    subsample=0.7808237001395564,
    scale_pos_weight=1
)

In [ ]:
clf3 = HistGradientBoostingClassifier(
    random_state=42,
    learning_rate=0.1,
    max_iter=100,
    class_weight='balanced',
)

In [ ]:
clf4 = LGBMClassifier(
    random_state=42,
    learning_rate=0.1,
    n_estimators=100,
    class_weight='balanced',
    verbosity=-1
)

In [ ]:
clf5= SVC(
    random_state=42,
    C=0.5522728891053806,
    gamma='auto',
    kernel='linear',
    class_weight='balanced'
)

In [ ]:
stack_model = StackingClassifier(
    estimators=[
        ('rf', clf1),
        ('hist_gb', clf3),
        ('lgbm',clf4),
#         ('svc',clf5)
    ],

    final_estimator=clf2,
    passthrough=True
)

In [ ]:
models = [clf1, clf2, clf3, clf4, clf5, stack_model]
model_names = ['RandomForest', 'XGBClassifier', 'HistGradientBoosting', 'LGBMClassifier','SVC', "StackingClassifier"]

models = [clf1, clf2, clf3, clf4, stack_model]
model_names = ['RandomForest', 'XGBClassifier', 'HistGradientBoosting', 'LGBMClassifier', "StackingClassifier"]

In [ ]:
def fit_predict_func(models, model_names, X_train, y_train, X_val, y_val):
    y_preds = []
    accuracies = []
    best_model = None
    best_accuracy = 0

    for model, name in zip(models, model_names):
        print(f"\nEvaluating {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        # Store predictions and accuracy
        y_preds.append((name, y_pred))
        accuracy = accuracy_score(y_val, y_pred)
        accuracies.append((name, accuracy))

        print(f"Validation Accuracy: {accuracy:.4f}")
        print(classification_report(y_val, y_pred))

        # Update the best model if the current one has higher accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

    return y_preds, accuracies, best_model

y_preds, accuracies, best_model = fit_predict_func(models, model_names, X_train, y_train, X_val, y_val)

# Classification Confusion Matrix

In [ ]:
import plotly.figure_factory as ff

def plot_confusion_matrices(y_preds, y_val):
    for name, y_pred in y_preds:

        cm = confusion_matrix(y_val, y_pred, labels=np.unique(y_val))
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
                    xticklabels=np.unique(y_val), 
                    yticklabels=np.unique(y_val))
        
        plt.title(f'Confusion Matrix for {name}')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()

plot_confusion_matrices(y_preds, y_val)

# Prediction and Submission

In [ ]:
test_pred = best_model.predict(X_test)

test_pred = le.inverse_transform(test_pred)

print("Best model used for predictions:")
print(best_model)

test_pred

In [ ]:
dfinal = {'ID':[i for i in range(1,len(test_pred)+1)],'Crime_Category':list(test_pred)}
dfinal = pd.DataFrame(dfinal)
dfinal.to_csv('submission.csv',index=False)
